In [4]:
# Importació de llibreries per a modelatge, processament de text i validació
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier # Classificador Gradient Boosting optimitzat


# Càrrega del conjunt de dades ' Newsgroups', una col·lecció de textos en anglès classificats per temes

dades = fetch_20newsgroups(subset='all', remove=())  # Es carrega el conjunt complet, sense eliminar cap part

# Es crea un DataFrame amb el text dels documents i la seva etiqueta numèrica
df = pd.DataFrame({
    'text': dades.data,
    'etiqueta_original': dades.target
})

# S’afegeix una columna amb el nom de la categoria textual corresponent a cada etiqueta
noms_categories = dades.target_names
df['tema'] = df['etiqueta_original'].apply(lambda x: noms_categories[x])

# Definició d'una variable binària "esport"
# Classes positives: categories relacionades amb esports i motors

categories_esport = ['rec.sport.baseball', 'rec.sport.hockey', 'rec.autos', 'rec.motorcycles']
df['esport'] = df['tema'].apply(lambda x: 1 if x in categories_esport else 0)

# Separació de les dades (X) i etiquetes binàries (y)

X = df['text'].values
y = df['esport'].values

# Definició d’una llista de paraules buides personalitzada
# Aquestes paraules seran ignorades durant la vectorització

stopwords = [
    'as', 'an', 'the', 'in', 'on', 'at', 'to', 'of', 'and', 'or',
    'is', 'it', 'for', 'with', 'that', 'this', 'was', 'be',
    'are', 'were', 'been', 'from', 'by', 'about', 'into', 'out',
    'up', 'down', 'over', 'under', 'then', 'than', 'so', 'but', 'not'
]

In [7]:

i=0# Definició del rang de valors per al paràmetre k (nombre de components SVD)
# i inicialització de la llista per emmagatzemar els resultats

valors_k = [100]  # Nombre de components latents a conservar amb SVD (LSA)
resultats = []    # Llista que contindrà el rendiment mitjà per a cada valor de k

# Configuració de la validació creuada estratificada amb 5 particions
# Es manté la proporció de classes entre entrenament i test en cada fold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Iteració principal per cada valor de components (k) a aplicar amb SVD

for k in valors_k:
    puntuacions = []  # Llista per desar els valors AUC-ROC de cada fold

    for train_idx, test_idx in cv.split(X, y):
        # Separació de les dades en conjunts d'entrenament i test

        X_train_raw, X_test_raw = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Vectorització del text mitjançant TF-IDF
        # S’ajusta només sobre l’entrenament per evitar fuita d'informació

        tfidf = TfidfVectorizer(max_features=10000)
        X_train_tfidf = tfidf.fit_transform(X_train_raw)
        X_test_tfidf = tfidf.transform(X_test_raw)

        # Reducció de dimensionalitat amb Truncated SVD (LSA)
        # Es redueix la representació TF-IDF a k dimensions latents

        svd = TruncatedSVD(n_components=k, random_state=42)
        X_train_svd = svd.fit_transform(X_train_tfidf)
        X_test_svd = svd.transform(X_test_tfidf)

        # Entrenament del model XGBoost per classificació binària
        # Paràmetres:
        # - max_depth: profunditat màxima dels arbres
        # - n_estimators: nombre d'arbres en el model 
        # - use_label_encoder: desactivat per evitar advertiments antics
        # - eval_metric: mètrica utilitzada durant l’entrenament

        model = XGBClassifier(
            max_depth=6,
            n_estimators=500,
            use_label_encoder=False,
            eval_metric='logloss',
            random_state=42
        )
        model.fit(X_train_svd, y_train)
        print("Iteracio: "+str(i) )  # Missatge per indicar que una iteració ha finalitzat

        # Predicció de probabilitats i càlcul de la mètrica AUC-ROC
        # Aquesta mètrica mesura la capacitat del model per separar les classes

        probs = model.predict_proba(X_test_svd)[:, 1]
        auc = roc_auc_score(y_test, probs)
        puntuacions.append(auc)
        print("AUC-ROC: Fold " + str(i) +  ":" + str(auc))  # Impressió de la puntuació AUC del fold actual
        i=i+1
    # Un cop completats els 5 folds, es calcula la mitjana de les puntuacions
    # i s’emmagatzema el resultat per al valor de k actual

    resultats.append({
        'k_components': k,
        'auc_roc_mitjana': np.mean(puntuacions)
    })


c:\Users\tonie\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:13:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Iteracio: 0
AUC-ROC: Fold 0:0.9871638792482926


c:\Users\tonie\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:13:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Iteracio: 1
AUC-ROC: Fold 1:0.9904466834677931


c:\Users\tonie\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:13:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Iteracio: 2
AUC-ROC: Fold 2:0.9893480182615061


c:\Users\tonie\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:14:17] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Iteracio: 3
AUC-ROC: Fold 3:0.9931791906048912


c:\Users\tonie\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:14:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Iteracio: 4
AUC-ROC: Fold 4:0.9907309893289009


In [9]:
# Organitzem els resultats
df_resultats = pd.DataFrame(resultats)

# Mostrar la taula de resultats
print("\n Resultats de validació creuada (mitjana AUC-ROC per a cada valor de k=100):\n")
print(df_resultats.to_string(index=False, float_format="%.4f"))


 Resultats de validació creuada (mitjana AUC-ROC per a cada valor de k=100):

 k_components  auc_roc_mitjana
          100           0.9902
